In [1]:
%reload_ext autoreload
%autoreload 2

from geopandas import GeoDataFrame
import geopandas as gpd
from pathlib import Path
import numpy as np
import cv2
import matplotlib.pyplot as plt

# Fetch Data

# Distance Transform

In [ ]:
file_path = Path("RoadSectionLine_Jul2024/RoadSectionLine.shp")
data: GeoDataFrame = gpd.read_file(file_path)
data.plot()

In [3]:
line_data = data.explode()
line_data["coords"] = line_data.geometry.apply(lambda geom: list(geom.coords))

In [22]:
# Function to convert shapefile geometries to OpenCV contours
def road_shapefile_to_opencv_mat(coords, img_width, img_height):
    scale_x, scale_y = (
        img_width / data.total_bounds[2],
        img_height / data.total_bounds[3],
    )
    image = np.zeros((img_height, img_width), dtype=np.uint8)

    for polyline in coords:
        pts = np.array(polyline)
        pts[:, 0] *= scale_x
        pts[:, 1] *= scale_y
        image = cv2.polylines(
            image, np.int32([pts]), isClosed=False, color=255, thickness=1
        )

    return image


image = road_shapefile_to_opencv_mat(
    line_data["coords"], img_width=3200, img_height=3200
)

In [ ]:
cv2.imwrite("test.png", image)

In [23]:
dist = cv2.distanceTransform(cv2.bitwise_not(image), cv2.DIST_L2, 0)

In [ ]:
np.unique(dist)
plt.imshow(dist[2030:2200, 1255:1375], origin="lower")
plt.colorbar()

In [ ]:
ret, thresh = cv2.threshold(image, 127, 255, cv2.THRESH_BINARY)

np.unique(thresh)

In [ ]:
image.shape